In [21]:
TRAIN_URL = 'http://storage.googleapis.com/tf-datasets/titanic/train.csv'
TEST_URL = 'http://storage.googleapis.com/tf-datasets/titanic/test.csv'

import tensorflow as tf
from tensorflow import keras

TRAIN_URL = 'http://storage.googleapis.com/tf-datasets/titanic/train.csv'
TEST_URL =  'http://storage.googleapis.com/tf-datasets/titanic/eval.csv'

train = keras.utils.get_file("train.csv", TRAIN_URL)
test = keras.utils.get_file("test.csv", TEST_URL)

# print the paths where these files are stored locally
print(train)
print(test)

# does not load the full file in memory, 
# but rather reads in batches when .take(n) is called
# make_csv_dataset has other interesting parameters, such as to recover from errors, 
# or fill the missing data with a default value
train = tf.data.experimental.make_csv_dataset(
    train, 10, 
    label_name="survived",
    shuffle=False,
    num_epochs=1)

def inspect_batch(dataset):
    
    # read the first batch
    for batch, label in dataset.take(1):
        
        # in batch is everything else, as a dictionary of columns
        for k, v in batch.items():
            print(f'{k} : {v.numpy()}')
            
        # it was imported with label (make_csv_dataset invocation)
        print(f'survived : {label.numpy()}')
        
inspect_batch(train)
[(train_features, label_batch)] = train.take(1)

/Users/alexandrugris/.keras/datasets/train.csv
/Users/alexandrugris/.keras/datasets/test.csv
sex : [b'male' b'female' b'female' b'female' b'male' b'male' b'female' b'female'
 b'female' b'male']
age : [22. 38. 26. 35. 28.  2. 27. 14.  4. 20.]
n_siblings_spouses : [1 1 0 1 0 3 0 1 1 0]
parch : [0 0 0 0 0 1 2 0 1 0]
fare : [ 7.25   71.2833  7.925  53.1     8.4583 21.075  11.1333 30.0708 16.7
  8.05  ]
class : [b'Third' b'First' b'Third' b'First' b'Third' b'Third' b'Third' b'Second'
 b'Third' b'Third']
deck : [b'unknown' b'C' b'unknown' b'C' b'unknown' b'unknown' b'unknown'
 b'unknown' b'G' b'unknown']
embark_town : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Queenstown'
 b'Southampton' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone : [b'n' b'n' b'y' b'n' b'y' b'n' b'n' b'n' b'n' b'y']
survived : [0 1 1 1 0 0 1 1 1 0]
OrderedDict([('sex', <tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'male', b'female', b'female', b'female', b'male', b'male',
     

In [19]:
# we will use keras preprocessing layers to process the following features
numeric_features = ['age', 'fare', 'n_siblings_spouses', 'parch'] # to be normalized
categorical_features = ['class', 'deck', 'embark_town', 'sex', 'alone'] # to be one-hot encoded

def get_normalization_layer(ds, col):
    layer = keras.layers.Normalization(axis=None)
    ds = ds.map(lambda x, y: x[col])
    layer.adapt(ds)
    return layer

def get_categorical_layer(ds, col):
    return None

preprocessing_inputs = []

for nf in numeric_features:
    preprocessing_inputs.append(get_normalization_layer(train, nf))
    print(preprocessing_inputs[-1](train_features[nf]))

for cf in categorical_features:
    preprocessing_inputs.append(get_categorical_layer(train, cf))


2022-06-05 22:47:43.511816: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-05 22:47:43.527093: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


tf.Tensor(
[-0.6104155   0.66939646 -0.29046255  0.4294317  -0.13048604 -2.2101805
 -0.2104743  -1.2503215  -2.050204   -0.77039206], shape=(10,), dtype=float32)


2022-06-05 22:47:43.723149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-05 22:47:43.738188: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


tf.Tensor(
[-0.49740294  0.67635316 -0.48502994  0.3430459  -0.47525433 -0.24398515
 -0.42622048 -0.07908852 -0.32418063 -0.48273864], shape=(10,), dtype=float32)


2022-06-05 22:47:43.938763: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-05 22:47:43.955414: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


tf.Tensor(
[ 0.39519805  0.39519805 -0.47423768  0.39519805 -0.47423768  2.1340694
 -0.47423768  0.39519805  0.39519805 -0.47423768], shape=(10,), dtype=float32)
tf.Tensor(
[-0.47905254 -0.47905254 -0.47905254 -0.47905254 -0.47905254  0.78298926
  2.045031   -0.47905254  0.78298926 -0.47905254], shape=(10,), dtype=float32)


2022-06-05 22:47:44.183529: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-05 22:47:44.198812: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
